In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
cd drive/MyDrive/TargetClassify/

In [ ]:
ls

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import time
import os

In [ ]:
# load data
data_dir = './DataSet/'
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop((100,100)),
        transforms.RandomRotation(5),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        # transforms.Grayscale(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: datasets.ImageFolder(
                    os.path.join(data_dir, x),
                    data_transforms[x]) for x in ['train', 'test']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}

dataloders = {x: torch.utils.data.DataLoader(image_datasets[x],batch_size=64, 
                                shuffle=True,
                                num_workers=4) 
                                for x in ['train', 'test']}


In [ ]:
flag=0
for data in dataloders['train']:
  if flag<1:
    attr,label=data
    print(attr[0],label)
    flag+=1
    

In [ ]:
#读取pytorch自带的resnet50模型,因为使用了预训练模型，所以会自动下载模型参数
ResNet=models.resnet50(pretrained=True)
class_num = 10
channel_in = ResNet.fc.in_features#获取fc层的输入通道数
ResNet.fc=nn.Sequential(
        nn.Linear(channel_in, 256),
        nn.Dropout(0.5),
        # nn.BatchNorm1d(256),
        nn.ReLU(inplace=True),
        # nn.Dropout(0.3),
        nn.Linear(256,class_num),
        nn.Softmax(dim=1)
    )

#对于模型的每个权重，使其不进行反向传播，即固定参数
for param in ResNet.parameters():
  param.requires_grad=False
        
#但是参数全部固定了，也没法进行学习，所以我们不固定最后一层，即全连接层fc
for param in ResNet.fc.parameters():
    param.requires_grad = True


In [ ]:
#对于模型的前cnt个权重，使其不进行反向传播，即固定参数
cnt=0
for param in ResNet.parameters():
  if cnt<147:
    param.requires_grad = False
    cnt+=1
    

In [ ]:
# 输出参数是否固定
for k,v in ResNet.named_parameters():
  print('{}:{}'.format(k,v.requires_grad))

# 输出网络结果
# for k in ResNet.named_children():
#   print(k)


In [ ]:
# 加载新模型ResNet
use_gpu=torch.cuda.is_available()
device = torch.device('cuda')
model=ResNet.to(device)

# 加载之前的模型
# model = torch.load('model.pth')
# model=model.to(device)

# cnt=0
# for param in model.parameters():
#   if cnt>135:
#     param.requires_grad = True
#     cnt+=1


In [ ]:
#filter()函数过滤掉parameters中requires_grad=Fasle的参数
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()),#重要的是这一句
                        lr=0.001,momentum=0.9)
criterion = nn.CrossEntropyLoss()


In [ ]:
# 每step_size个epochs后lr=lr*gamma
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)


In [ ]:
# 开始训练&测试，记录测试集准确率最高时的模型参数
num_epochs=400
scheduler=exp_lr_scheduler
since = time.time()
best_model_wts = model.state_dict()
best_acc = 0.0

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'test']:
        if phase == 'train':
            model.train(True)  # Set model to training mode
        else:
            model.train(False)  # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0.0

        # Iterate over data.
        for data in dataloders[phase]:
            # get the inputs
            inputs, labels = data
            # wrap them in Variable
            if use_gpu:
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda())
            else:
                inputs, labels = Variable(inputs), Variable(labels)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)

            # backward + optimize only if in training phase
            if phase == 'train':
                loss.backward()
                optimizer.step()
                scheduler.step()

            # statistics
            running_loss += loss.item()
            running_corrects += torch.sum(preds == labels.data).to(torch.float32)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects / dataset_sizes[phase]

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            phase, epoch_loss, epoch_acc))

        # deep copy the model
        if phase == 'test' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = model.state_dict()

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
print('Best val Acc: {:4f}'.format(best_acc))

# load best model weights
model.load_state_dict(best_model_wts)


In [ ]:
# 模型保存
torch.save(model, 'model_1.pth')